In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv.zip')
credits = pd.read_csv('tmdb_5000_credits.csv.zip')

In [ ]:
movies.head().shape

In [ ]:
credits.head().shape

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
movies.shape

In [ ]:
movies.head(1)

##### use data columns

In [ ]:
# genres
# id
# keywords
# title
# overview
# cast                  
# crew
movies = movies[['movie_id','title','keywords','genres','cast','crew','overview']]

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

## set the data of genres columns 

In [ ]:
movies.iloc[0].genres

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]

# ['Action ,Adventure ,Fantasy,Science Fiction]

In [ ]:
import ast
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l    

In [ ]:
#import ast
#ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies['cast']        #actors

In [ ]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3 :
            l.append(i['name'])
            counter+1
        else:
            break
    return l    

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head(1)

In [ ]:
movies['crew']

In [ ]:
import ast
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l    

In [ ]:
movies['crew'].apply(fetch_director)

movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" "," ") for i in x])

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(1)

In [ ]:
movies['tags'] =  movies['overview'] + movies['keywords'] + movies['genres'] + movies['cast'] + movies['crew']

In [ ]:
movies.head(1)

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x :" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags']

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

In [ ]:
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
feature_names = cv.get_feature_names_out()
print(feature_names)

In [ ]:
!pip install nltk

In [ ]:
import nltk  

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return ' '.join(y)
                             

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags']

In [ ]:
stem('in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d action adventur fantasi sciencefict samworthington zoesaldana sigourneyweav stephenlang michellerodriguez giovanniribisi joeldavidmoor cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyt scottlawr kellykilgour jamespatrickpitt seanpatrickmurphi peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurri lukehawk woodyschultz petermensah soniay jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibbi jodielandau julielamm cullenb.madden josephbradymadden frankietorr austinwilson sarawilson tamicawashington-mil lucybri nathanmeist gerryblair matthewchamberlain paulyat wraywilson jamesgaylyn melvinlenoclarkiii carvonfutrel brandonjelk micahmoch hanniyahmuhammad christophernolen christaoliv aprilmariethoma bravitaa.threatt colinbleasd mikebodnar mattclayton nicoledionn jamieharrison allanhenri anthonyingrub ashleyjefferi deanknowsley josephmika-hunt terrynotari kaipantano loganpithi stuartpollock raja garethruck rhiansheehan t.j.storm jodietaylor aliciavela-bailey richardwhitesid nikiezambo juleneren')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
#sorted(list(enumerate(similarity[0])),reverse=True ,key=lambda x:x[1])

In [ ]:
#sorted distances
sorted(list(enumerate(similarity[0])) ,reverse=True,key = lambda x : x[1] )[1:6]     #movies distace from 0th path

In [ ]:
def recommend(movie):                         #agar koi movie dii jye too uss ka sabse paele index nikalna pade gya data ke andar
    movie_index = new_df[new_df['title'] == movie].index[0]
    ditances = similarity[movie_index]
    movies_list = sorted(list(enumerate(ditances)) ,reverse=True,key = lambda x : x[1] )[1:6]


    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        print()



    

In [ ]:
recommend("Batman Begins")

In [ ]:
new_df.iloc[1214].title

In [ ]:
new_df[new_df['title'] == "Avatar"].index[0]

In [ ]:
new_df.head(0)

In [ ]:
import pickle 

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [1]:
jupyter nbconvert --validate your_notebook.ipynb



SyntaxError: invalid syntax (640951514.py, line 1)

In [3]:
jupyter nbconvert --validate Movie-Recommend-System.ipynb


SyntaxError: invalid syntax (2998803418.py, line 1)